In [ ]:
import pandas as pd
import numpy as np
import re

def extract_columns_and_sort(input_file, output_file, sort_column):
    df = pd.read_excel(input_file)

    print("Available columns in the dataset:")
    print(df.columns)

    df.columns = df.columns.str.strip()

    ##### checking status as operating & and technology type also solar
    missing_columns = [col for col in ['Country/Area', 'Status', 'Technology Type'] if col not in df.columns]
    if missing_columns:
        print(f"Warning: The following columns are missing from the DataFrame: {missing_columns}")
    
  
    df_filtered = df[(df['Country/Area'] == 'India') & 
                     (df['Status'] == 'operating') & 
                     (df['Technology Type'] != 'Onshore')].copy()  
 
    #### creating new columns for the next process          
    df_filtered['ppa_capacity'] = None
    df_filtered['unit_price'] = None
    df_filtered['degradation_factor'] = None
    df_filtered['aux_power'] = None

    ###  checking null values by using np method and apply method
    df_filtered['start_year_is_empty'] = np.where(df_filtered['Start year'].isnull(), 'Yes', 'No')
    df_filtered['owner_is_empty'] = np.where(df_filtered['Owner'].isnull(), 'Yes', 'No')
    df_filtered['ppa_tenure_is_empty'] = np.where(df_filtered['PPA Tenure'].isnull(), 'Yes', 'No')
    df_filtered['ppa_offtaker_is_empty'] = np.where(df_filtered['PPA Offtaker'].isnull(), 'Yes', 'No')

    # df_filtered['start_year_is_empty'] = df_filtered['Start year'].apply(lambda x: 'Yes' if pd.isnull(x) else 'No')
    # df_filtered['owner_is_empty'] = df_filtered['Owner'].apply(lambda x: 'Yes' if pd.isnull(x) else 'No')
    # df_filtered['ppa_tenure_is_empty'] = df_filtered['PPA Tenure'].apply(lambda x: 'Yes' if pd.isnull(x) else 'No')
    # df_filtered['ppa_offtaker_is_empty'] = df_filtered['PPA Offtaker'].apply(lambda x: 'Yes' if pd.isnull(x) else 'No')


    ### modifying phase name from -- to 1 
    if 'Phase Name' in df.columns:
        df_filtered['phase_name'] = df['Phase Name'].replace('--', 1)  
    else:
        print("Warning: 'Phase Name' column not found in the dataset.")
        df_filtered['phase_name'] = None

    
    cols = list(df_filtered.columns)
    

    ### getting location by using get_loc method 
    start_year_index = df_filtered.columns.get_loc('Start year')
    owner_index = df_filtered.columns.get_loc('Owner')
    ppa_tenure_index = df_filtered.columns.get_loc('PPA Tenure')
    ppa_offtaker_index = df_filtered.columns.get_loc('PPA Offtaker')

    ### removing & inserting the new column by using specific location
    cols.remove('start_year_is_empty')
    cols.insert(start_year_index + 1, 'start_year_is_empty')

    cols.remove('owner_is_empty')
    cols.insert(owner_index + 1, 'owner_is_empty')


    cols.remove('ppa_tenure_is_empty')
    cols.insert(ppa_tenure_index + 1, 'ppa_tenure_is_empty')

    cols.remove('ppa_offtaker_is_empty')
    cols.insert(ppa_offtaker_index + 1, 'ppa_offtaker_is_empty')

    cols.append('phase_name')

   
    df_filtered = df_filtered[cols]
 
    #### removing unwanted column from the uncleaned data to new data
    unwanted_columns = [
        'Unnamed: 0', 'Date Last Researched', 'Project Name in Local Language / Script',
        'Retired year', 'Operator', 'Operator Name in Local Language / Script', 
        'Owner Name in Local Language / Script', 'Hydrogen', 'Other IDs (location)', 
        'Other IDs (unit/phase)'
    ]

    df_filtered = df_filtered.drop(columns=[col for col in unwanted_columns if col in df_filtered.columns])

    #### checking columns null value count
    null_counts = df_filtered.isnull().sum()
    
    columns_with_nulls = null_counts[null_counts > 0]

    if not columns_with_nulls.empty:
        print("Columns with null values:")
        print(columns_with_nulls)
    else:
        print("No columns with null values found.")

    ### replacing column name by using _ instead of '' (space)
    df_filtered.columns = df_filtered.columns.str.replace(' ', '_').str.lower()

    ## sorting the column by using snake case
    sort_column_snake_case = sort_column.lower().replace(' ', '_')
    
    df_sorted = df_filtered.sort_values(by=sort_column_snake_case)

    df_sorted.to_excel(output_file, index=False)
    
    ### checking unique owners
    unique_owners = df_filtered['owner'].nunique()
    print(f"There are {unique_owners} unique owners in the dataset from India.")
    print(f"Columns extracted, sorted by '{sort_column}', new columns added for conditions. Saved to {output_file}")


input_file = '/Users/sneha/Downloads/RE_technical_merged_2.xlsx'  
output_file = 'solar_RE_data_all_column.xlsx'


sort_column = 'Owner'


extract_columns_and_sort(input_file, output_file, sort_column)

In [ ]:

input_file = '/Users/sneha/notebook/nov14/solar_RE_data_all_column.xlsx'  
df = pd.read_excel(input_file)

### removing unwanted column 
columns_to_drop = ['phase_name1', 'phase_name2']
df.drop(columns=columns_to_drop, inplace=True)

## removing ppa_tariff 0 with null 
df.loc[df['ppa_tariff'] == 0, 'ppa_tariff'] = pd.NA
 
## cleaning the NA data from the plf_dq_1 column
if 'plf_dq_1' in df.columns:
    df['plf_dq_1'].replace(['Not Available', 'not Available', 'not available', 
                             'Not Found', 'Unknown', 'Not available', 'Not available '], 
                            pd.NA, inplace=True)

    ## removing % from the plf_dq_1 column
    df['plf_dq_1'] = df['plf_dq_1'].apply(lambda x: str(x).replace('%', '') if isinstance(x, str) else x)

    
    df['plf_dq_1'] = df['plf_dq_1'].apply(lambda x: pd.NA if isinstance(x, str) and (
        'More than P90 PLF estimation' in x or 
        'above degraded P90 levels' in x or 
        'P90' in x or  
        'Plant Load Factor: 0.5' in x) else x)

## cleaning capacity_rating column with null values
df['capacity_rating'] = df['capacity_rating'].replace(['unknown', 'Unknown'], pd.NA)

## removing ppa_tariff_dq_1 0 with null 
df.loc[df['ppa_tariff_dq_1'] == 0, 'ppa_tariff_dq_1'] = pd.NA

### spliting the ppa_tariff & ppa_tariff_dq_1 column like value,currency,unit
def extract_numeric_or_range(value):
    if pd.isna(value):
        return pd.NA
    
    range_match = re.search(r'(\d+(\.\d+)?)\s*-\s*(\d+(\.\d+)?)', str(value))
    if range_match:
        lower_value = float(range_match.group(1))
        upper_value = float(range_match.group(3))
        return (lower_value, upper_value)
    
    single_match = re.search(r'\b\d+(\.\d+)?\b', str(value))
    return float(single_match.group()) if single_match else pd.NA


def extract_inr_value(value):
    if pd.isna(value):
        return pd.NA
    value_str = str(value)
  
    if 'INR' in value_str and 'USD' in value_str:
        match = re.search(r'INR\s*([\d.]+)', value_str)
        return float(match.group(1)) if match else pd.NA
    inr_no_space_match = re.search(r'INR([\d.]+)', value_str)
    if inr_no_space_match:
        return float(inr_no_space_match.group(1))
    return extract_numeric_or_range(value)


def detect_currency(value):
    if pd.isna(value):
        return pd.NA
    value_str = str(value).upper()
    if any(currency in value_str for currency in ['INR', 'RS.', '₹', 'RS']):
        return 'INR'
    elif 'USD' in value_str or 'US$' in value_str:
        return 'USD'
    else:
        return pd.NA


def detect_unit(value):
    if pd.isna(value):
        return pd.NA
    value_str = str(value).lower()
    if any(unit in value_str for unit in ['/kwh', 'kwh', 'per kwh']):
        return 'kWh'
    elif any(unit in value_str for unit in ['per unit', '@ unit', "/ unit", "/unit"]):
        return 'unit'
    return pd.NA

### applying that value to column
df['ppa_tariff_dq1_value'] = df['ppa_tariff_dq_1'].apply(extract_inr_value)
df['ppa_tariff_dq1_currency'] = df['ppa_tariff_dq_1'].apply(detect_currency)
df['ppa_tariff_dq1_unit'] = df['ppa_tariff_dq_1'].apply(detect_unit)
df['ppa_tariff_value'] = df['ppa_tariff'].apply(extract_inr_value)
df['ppa_tariff_currency'] = df['ppa_tariff'].apply(detect_currency)
df['ppa_tariff_unit'] = df['ppa_tariff'].apply(detect_unit)


# df['ppa_tariff_final'] = df['ppa_tariff_dq1_value'].fillna(df['ppa_tariff_value'])

### validating plf columns ,checking the range its 18 to 25 if the value is morethan or less than this value just give null
def validate_plf(value):
    try:
      
        value_float = float(value)
        
        
        if 18 <= value_float <= 25:
            return value_float
        else:
            return ''  
    except (ValueError, TypeError):
        
        return ''

### creating final plf by using above condition
df['plf_final'] = df['plf_dq_1'].apply(validate_plf)

### checking the plf value also if plf_dq_1 crt range it takes that value elif it will check plf value if its correct it will take that or
## it will give space ''
def validate_plf_fallback(value, fallback_value):
    
    if value == '':
        
        return validate_plf(fallback_value)
    else:
        return value


df['plf_final'] = df.apply(lambda row: validate_plf_fallback(row['plf_final'], row['plf']), axis=1)


df = df.drop(columns=['value'], errors='ignore')

output_file = 'solar.xlsx' 
df.to_excel(output_file, index=False)

print("Columns dropped, specific values in 'plf_dq_1' replaced with NaN, specific phrases removed, 'ppa_tariff_final' created, 'plf_final' with valid values created, and file saved successfully.")


/var/folders/1s/f_42bgbj0x30lz24x3f4f2zw0000gn/T/ipykernel_5069/1205246945.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['plf_dq_1'].replace(['Not Available', 'not Available', 'not available',


Columns dropped, specific values in 'plf_dq_1' replaced with NaN, specific phrases removed, 'ppa_tariff_final' created, 'plf_final' with valid values created, and file saved successfully.


In [ ]:
import pandas as pd
import numpy as np
import re

file_path = '/Users/sneha/notebook/nov19-upload_db/solar.xlsx'
df = pd.read_excel(file_path)

### sorting the column by using owner & ppa_tariff_dq_1_value
df_sorted = df.sort_values(by=['owner', 'ppa_tariff_dq1_value'], ascending=[True, False])

## grouped the data by using project_name,phase_name and ppa_offtaker 
df_unique = df_sorted.drop_duplicates(subset=['project_name', 'phase_name', 'ppa_offtaker'], keep='first')

## changing the phase name value with 1,2,3 
df_unique['phase_name'] = df_unique['phase_name'].replace({'Unit 1': '1', 'Unit 2': '2', 'Unit 3': '3'})

### creating currency_unit by using currency and unit column for ppa_tarif and ppa_tariff_dq_1 column
def create_currency_unit_dq1(row):
    currency = row['ppa_tariff_dq1_currency']
    unit = row['ppa_tariff_dq1_unit']
    
    if pd.notna(currency) and pd.notna(unit):
        return f"{currency}/{unit}"
    elif pd.notna(currency):
        return currency
    elif pd.notna(unit):
        return unit
    else:
        return ""

df_unique['ppa_tariff_dq1_currency_unit'] = df_unique.apply(create_currency_unit_dq1, axis=1)

def create_currency_unit(row):
    currency = row['ppa_tariff_currency']
    unit = row['ppa_tariff_unit']
    
    if pd.notna(currency) and pd.notna(unit):
        return f"{currency}/{unit}"
    elif pd.notna(currency):
        return currency
    elif pd.notna(unit):
        return unit
    else:
        return ""
    

df_unique['ppa_tariff_currency_unit']=df_unique.apply(create_currency_unit,axis=1)

## checking the ppa_tariff_dq1_value is numeric 
df_unique['ppa_tariff_dq1_value'] = pd.to_numeric(df_unique['ppa_tariff_dq1_value'], errors='coerce')

### checking the value of tarrif & dq1 value 
def check_tariff_out_of_range(value, currency):
    if pd.notna(value):
       
        if pd.isna(currency):
            currency = 'INR'
        
       
        if currency == 'INR' and (value < 1.9 or value > 12.76):
            return 'Yes'
       
        elif currency == 'USD' and (value < 0.023 or value > 0.15):
            return 'Yes'
        
        else:
            return 'No'
    else:
        return '' 

df_unique['ppa_tariff_dq1_value_outof_range'] = df_unique.apply(
    lambda row: check_tariff_out_of_range(row['ppa_tariff_dq1_value'], row['ppa_tariff_dq1_currency']), axis=1
)


df_unique['ppa_tariff_value_outof_range'] = df_unique.apply(
    lambda row: check_tariff_out_of_range(row['ppa_tariff_value'], row['ppa_tariff_currency']), axis=1
)


### checking plf value with specific range for creating yes or no column
def check_plf_out_of_range(plf_value):
    if pd.isna(plf_value):
        return ''
    
    if isinstance(plf_value, str) and '-' in plf_value:
        try:
            lower, upper = map(float, plf_value.split('-'))
            if lower < 18 or upper > 25:
                return 'Yes'
            else:
                return 'No'
        except ValueError:
            return ''
    
    try:
        plf_value = float(plf_value)
        if plf_value < 18 or plf_value > 25:
            return 'Yes'
        else:
            return 'No'
    except ValueError:
        return ''

df_unique['plf_dq1_out_of_range'] = df_unique['plf_dq_1'].apply(check_plf_out_of_range)
df_unique['plf_final_out_of_range'] = df_unique['plf_final'].apply(check_plf_out_of_range)

### creating ppa_tariff final column by using tariff and dq1 column
def create_ppa_tariff_final(row):
    ppa_tariff_dq1_value = row['ppa_tariff_dq1_value']
    ppa_tariff_value = row['ppa_tariff_value']
    ppa_tariff_dq1_currency = row['ppa_tariff_dq1_currency']
    ppa_tariff_currency=row['ppa_tariff_currency']
    

    if check_tariff_out_of_range(ppa_tariff_dq1_value, ppa_tariff_dq1_currency) == 'No':
        return ppa_tariff_dq1_value
  
    elif check_tariff_out_of_range(ppa_tariff_value, ppa_tariff_currency) == 'No':
        return ppa_tariff_value
    else:
        return None  

df_unique['ppa_tariff_final'] = df_unique.apply(create_ppa_tariff_final, axis=1)


df_unique['ppa_tariff_final_empty'] = df_unique['ppa_tariff_final'].apply(lambda x: 'Yes' if pd.isna(x) else 'No')

### checking null values for tariff and plf final columns
df_unique['ppa_tariff_dq_1_empty'] = df_unique['ppa_tariff_dq_1'].apply(lambda x: 'Yes' if pd.isna(x) else 'No')
df_unique['plf_dq_1_empty'] = df_unique['plf_dq_1'].apply(lambda x: 'Yes' if pd.isna(x) else 'No')
df_unique['ppa_tariff_final_empty'] = df_unique['ppa_tariff_final'].apply(lambda x: 'Yes' if pd.isna(x) else 'No')
df_unique['plf_final_empty'] = df_unique['plf_final'].apply(lambda x: 'Yes' if pd.isna(x) else 'No')


output_file_path = 'solar3.xlsx'
df_unique.to_excel(output_file_path, index=False)

print(f"ppa_tariff_final column created, and file saved as {output_file_path}")


/var/folders/1s/f_42bgbj0x30lz24x3f4f2zw0000gn/T/ipykernel_1621/2634232598.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['phase_name'] = df_unique['phase_name'].replace({'Unit 1': '1', 'Unit 2': '2', 'Unit 3': '3'})
/var/folders/1s/f_42bgbj0x30lz24x3f4f2zw0000gn/T/ipykernel_1621/2634232598.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['ppa_tariff_dq1_currency_unit'] = df_unique.apply(create_currency_unit_dq1, axis=1)
/var/folders/1s/f_42bgbj0x30lz24x3f4f2zw0000gn/T/ipykerne

ppa_tariff_final column created, and file saved as solar3.xlsx


In [ ]:
input_file_path = '/Users/sneha/notebook/nov19-upload_db/solar3.xlsx'  
output_file = 'solar_output_file3.xlsx'

### finall step to remove all null rows from the dataframe 
df = pd.read_excel(input_file_path)

df = df[(df['start_year_is_empty'] != 'Yes') & 
        (df['technology_type'] != 'Assumed PV') & 
        (df['ppa_tariff_final_empty'] != 'Yes') & 
        (df['plf_final_empty'] != 'Yes') & 
        (df['ppa_offtaker_is_empty']!= 'Yes')
        ]


df.to_excel(output_file, index=False)

print("Rows with 'start_year_is_empty' == 'Yes', 'technology_type' == 'Assumed PV', or 'ppa_tariff_final_empty' == 'Yes' have been removed and saved to the output file.")



Rows with 'start_year_is_empty' == 'Yes', 'technology_type' == 'Assumed PV', or 'ppa_tariff_final_empty' == 'Yes' have been removed and saved to the output file.
